basis GPU configuration

 libraries and env configuration 


In [1]:
#packages from tensor flow
import tensorflow as Tf

# tensor flow for optimizing the model 


# basis packages 
import os as os 
import matplotlib.pyplot as plt
import numpy as np
import scipy 
import ssl

In [2]:
# Comprobación y configuración segura de GPU para TensorFlow (en español)
# Esta celda: lista GPUs, activa memory growth, muestra info de build y hace una prueba rápida
import sys
import numpy as np
import tensorflow as Tf
import os

# Información del entorno Python y TensorFlow
print('Python:', sys.version.splitlines()[0])
print('TensorFlow version:', Tf.__version__)
print('TF built with CUDA:', Tf.test.is_built_with_cuda())

# Verificar variables de entorno CUDA (útil para diagnóstico)
cuda_visible = os.environ.get('CUDA_VISIBLE_DEVICES', 'No establecido')
print('\nCUDA_VISIBLE_DEVICES:', cuda_visible)

# Listar GPUs físicas
gpus = Tf.config.list_physical_devices('GPU')
print('\nGPUs físicas encontradas:', len(gpus))
for i, g in enumerate(gpus):
    print(f'  GPU {i}:', g)

if len(gpus) == 0:
    print('\nNo se detectaron GPUs en TensorFlow. Verifica:')
    print('1. nvidia-smi muestra la GPU')
    print('2. Las DLLs de CUDA están en el PATH')
    print('3. La versión de CUDA es compatible con TF 2.10')

# Activar memory growth (debe hacerse antes de crear tensores)
for gpu in gpus:
    try:
        Tf.config.experimental.set_memory_growth(gpu, True)
        print('\nMemory growth activado para', gpu)
    except Exception as e:
        print('\nNo se pudo activar memory growth para', gpu, '->', e)

# Mostrar información del build (CUDA/cuDNN si está disponible)
try:
    info = Tf.sysconfig.get_build_info()
    print('\nInformación del build de TF:')
    print('  cuda_version:', info.get('cuda_version', 'No disponible'))
    print('  cudnn_version:', info.get('cudnn_version', 'No disponible'))
except Exception:
    pass

# Prueba rápida: multiplicación pequeña en GPU:0 (si existe)
if len(gpus) > 0:
    try:
        with Tf.device('/GPU:0'):
            print('\nEjecutando prueba en GPU...')
            a = Tf.constant(np.random.randn(1000, 1000), dtype=Tf.float32)
            b = Tf.constant(np.random.randn(1000, 1000), dtype=Tf.float32)
            c = Tf.matmul(a, b)
            # Forzar ejecución
            _ = c.numpy()
        print('✅ Multiplicación de matrices completada en GPU')
        print('   Shape del resultado:', c.shape)
    except Exception as e:
        print('❌ La prueba GPU falló:', e)
else:
    print('\nSe omite la prueba GPU porque no se detectó GPU.')

Python: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]
TensorFlow version: 2.10.0
TF built with CUDA: True

CUDA_VISIBLE_DEVICES: No establecido

GPUs físicas encontradas: 1
  GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Memory growth activado para PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Información del build de TF:
  cuda_version: 64_112
  cudnn_version: 64_8

Ejecutando prueba en GPU...
✅ Multiplicación de matrices completada en GPU
   Shape del resultado: (1000, 1000)


In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2' # Suppress TensorFlow logging (1)	

## Verificación del Entorno
Primero verificamos que estamos usando el entorno correcto y tiene TensorFlow-GPU

In [4]:
import sys
import tensorflow as tf

# Verificar entorno Python
print("Python executable:", sys.executable)
print("Python version:", sys.version.splitlines()[0])

# Verificar TensorFlow y GPU
print("\nTensorFlow version:", tf.__version__)
print("GPU disponible:", "Sí" if tf.config.list_physical_devices('GPU') else "No")

# Mostrar información CUDA
print("\nCUDA built:", tf.test.is_built_with_cuda())
try:
    print("CUDA version:", tf.sysconfig.get_build_info()['cuda_version'])
except:
    print("CUDA version: No disponible")

Python executable: c:\Users\mateo\AppData\Local\Programs\Python\Python39\python.exe
Python version: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]

TensorFlow version: 2.10.0
GPU disponible: Sí

CUDA built: True
CUDA version: 64_112


## Verificar versión de TensorFlow y configuración

In [5]:
import tensorflow as tf
import sys

# Información detallada de versiones
print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version)

# Información de compilación
print("\nCompilado con CUDA:", tf.test.is_built_with_cuda())
print("Compilado con ROCm:", tf.test.is_built_with_rocm())

# Información detallada del build
build_info = tf.sysconfig.get_build_info()
print("\nInformación detallada del build:")
for key, value in build_info.items():
    print(f"{key}: {value}")

# Dispositivos disponibles
print("\nDispositivos disponibles:")
for device in tf.config.list_physical_devices():
    print(device)

TensorFlow version: 2.10.0
Python version: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]

Compilado con CUDA: True
Compilado con ROCm: False

Información detallada del build:
cpu_compiler: C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/HostX64/x64/cl.exe
cuda_compute_capabilities: ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']
cuda_version: 64_112
cudart_dll_name: cudart64_112.dll
cudnn_dll_name: cudnn64_8.dll
cudnn_version: 64_8
is_cuda_build: True
is_rocm_build: False
is_tensorrt_build: False
msvcp_dll_names: msvcp140.dll,msvcp140_1.dll
nvcuda_dll_name: nvcuda.dll

Dispositivos disponibles:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Verificar versiones de CUDA
Comprobamos tanto la versión del sistema como la que usa TensorFlow

In [6]:
import os
import subprocess
import tensorflow as tf

def get_nvidia_smi_output():
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        return result.stdout
    except:
        return "No se pudo ejecutar nvidia-smi"

# 1. Versión CUDA del sistema
print("=== CUDA del Sistema ===")
nvidia_output = get_nvidia_smi_output()
if "CUDA Version" in nvidia_output:
    cuda_line = [line for line in nvidia_output.split('\n') if "CUDA Version" in line][0]
    print(cuda_line.strip())
else:
    print("No se pudo detectar la versión de CUDA del sistema")

# 2. Variables de entorno CUDA
print("\n=== Variables de entorno CUDA ===")
cuda_path = os.environ.get('CUDA_PATH', 'No establecido')
cuda_home = os.environ.get('CUDA_HOME', 'No establecido')
print(f"CUDA_PATH: {cuda_path}")
print(f"CUDA_HOME: {cuda_home}")

# 3. Versión CUDA que usa TensorFlow
print("\n=== CUDA en TensorFlow ===")
print(f"TensorFlow compilado con CUDA: {tf.test.is_built_with_cuda()}")
try:
    build_info = tf.sysconfig.get_build_info()
    print(f"CUDA version (TF build): {build_info.get('cuda_version', 'No disponible')}")
    print(f"cuDNN version (TF build): {build_info.get('cudnn_version', 'No disponible')}")
except:
    print("No se pudo obtener información del build de TensorFlow")

=== CUDA del Sistema ===
| NVIDIA-SMI 581.29                 Driver Version: 581.29         CUDA Version: 13.0     |

=== Variables de entorno CUDA ===
CUDA_PATH: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.5
CUDA_HOME: No establecido

=== CUDA en TensorFlow ===
TensorFlow compilado con CUDA: True
CUDA version (TF build): 64_112
cuDNN version (TF build): 64_8
